# Forecasting Outflow 

## (1) Using forecasted inflow and satellite observed rule curve (∆S/∆Smax)

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from ipywidgets import interact, interactive, fixed, interact_manual, Layout
import ipywidgets as widgets
from collections import OrderedDict
from scipy.interpolate import interp1d

pd.set_option("max_colwidth", 50)

# Clip time-series
def clip_ts(*tss):
    mint = max([min(ts.index) for ts in tss])
    maxt = min([max(ts.index) for ts in tss])

    # clipped_tss = [ts[mint:maxt] for ts in tss]
    clipped_tss = [ts.loc[(ts.index>=mint)&(ts.index<=maxt)] for ts in tss]

    return clipped_tss

# From GRanD
Smaxes = {
    'Sirindhorn': 2000 * 1e6, # m3
    'Lam_Pao': 1980 * 1e6,
    'Ubol_Ratana': 2300 * 1e6
}

def get_rc(r):
    rc_fn = f"../data/rc/{r}.txt"
    rc = pd.read_csv(rc_fn)
    
    return rc

def get_inflow(r):
    inflow_fn = f"../data/forecasted_inflow/{r}.day"
    
    inflow = pd.read_csv(
        inflow_fn, 
        sep=r"\s+", 
        names=['year', 'month', 'day', 'streamflow'],
        parse_dates=[['year', 'month', 'day']]
    ).rename({"year_month_day": "date", 'streamflow': 'inflow [ft3/s]'}, axis=1).set_index('date')

    inflow['inflow [m3/s]'] = inflow['inflow [ft3/s]'] * 0.028316847
    inflow['I [m3]'] = inflow['inflow [m3/s]'] * 24*60*60
    
    return inflow

def get_aec(r):
    aec = pd.read_csv(f"../data/aec/{r}.csv")
    
    return aec

def get_sarea(r):
    sarea_path = f"../data/sarea/{r}.csv"
    sarea = pd.read_csv(sarea_path, parse_dates=['date']).set_index('date')[['area']].rename({'area': 'sarea'}, axis=1).dropna()
    
    return sarea

def get_dels(r):
    aec = get_aec(r)
    sarea = get_sarea(r)
    
    balance = sarea[['sarea']].rename({'sarea': 'sarea [km2]'}, axis=1)
    balance['sarea [m2]'] = balance['sarea [km2]'] * 1e6
    balance['h [m]'] = np.interp(balance['sarea [km2]'], aec['CumArea'], aec['Elevation'])
    balance['dels [m3]'] = (balance['h [m]']-balance['h [m]'].shift(1))*(balance['sarea [m2]'] + balance['sarea [m2]'].shift(1))/2

    return balance[['dels [m3]']]

def adder(dates, dataset):
    minmaxdates = dates.index[0], dates.index[-1]

    return dataset[minmaxdates[0]:minmaxdates[1]].sum()


def get_balance_nowcast(r):
    inflow = get_inflow(r)
    sarea = get_sarea(r)
    aec = get_aec(r)
    
    inflow_nowcast, sarea_nowcast = clip_ts(inflow, sarea)
    
    balance = sarea[['sarea']].rename({'sarea': 'sarea [km2]'}, axis=1)
    balance['sarea [m2]'] = balance['sarea [km2]'] * 1e6
    balance['h [m]'] = np.interp(balance['sarea [km2]'], aec['CumArea'], aec['Elevation'])
    balance['dels [m3]'] = (balance['h [m]']-balance['h [m]'].shift(1))*(balance['sarea [m2]'] + balance['sarea [m2]'].shift(1))/2
    
    balance['I [m3]'] = balance['sarea [km2]'].rolling(2).apply(lambda x: adder(x, inflow['I [m3]']))
    balance['O [m3]'] = balance['I [m3]'] - balance['dels [m3]']
    balance.loc[balance['O [m3]']<0, 'O [m3]'] = 0

    return balance
    
def get_dels_forecast(r, start_date, lead_time=15, freq=5):
    dates = pd.date_range(start_date, freq=f"{freq}d", periods=lead_time/freq + 1)
    Smax= Smaxes[r]

    rc = get_rc(r)

    years = list(OrderedDict.fromkeys([d.year for d in dates]))
    
    # pad the years list with two adjacent years for the edge cases of first and last 15 days of the year
    # Computationally cheap to just do the calculations for extra two years
    years.insert(0, years[0]-1)
    years.append(years[-1]+1)
    
    rcs_stretched = []
    
    for year in years:
        rc_stretched = rc.copy()
        rc_stretched['date_synthetic'] = [pd.to_datetime(f"{year}-{d:02}-15") for d in rc_stretched['Month']]
        rc_stretched.set_index('date_synthetic', inplace=True)
        rcs_stretched.append(rc_stretched)

    rcs_stretched = pd.concat(rcs_stretched)
    # rcs_stretched['S/Smax'].plot(marker='.')

    f = interp1d([d.to_pydatetime().timestamp() for d in rcs_stretched.index], rcs_stretched['S/Smax'])

    forecast_balance = pd.DataFrame({"date": dates})
    s_smaxes = []
    for d in dates:
        s_smaxes.append(f(d.to_pydatetime().timestamp()))
    forecast_balance['S/Smax'] = s_smaxes
    forecast_balance['S'] = forecast_balance['S/Smax']*Smax
    
    forecast_balance['dels [m3]'] = forecast_balance['S'].diff()

    return forecast_balance.set_index('date')[['dels [m3]']]

def get_forecast_balance(r, lead_time=15):
    inflow = get_inflow(r)
    dels_nowcast = get_dels(r)

    dels_nowcast['nowcast/forecast'] = 'nowcast'

    dels_forecast = get_dels_forecast(r, dels_nowcast.index.values[-1], lead_time=lead_time).dropna()
    dels_forecast['nowcast/forecast'] = 'forecast'

    dels = pd.concat([dels_nowcast, dels_forecast])

    balance = dels.copy()

    balance['I [m3]'] = balance['dels [m3]'].rolling(2).apply(lambda x: adder(x, inflow['I [m3]']))
    balance['O [m3]'] = balance['I [m3]'] - balance['dels [m3]']
    balance.loc[balance['O [m3]']<0, 'O [m3]'] = 0
    balance['O [m3/day]'] = balance['O [m3]']/balance.index.to_series().diff().dt.days
    balance['dels [m3/day]'] = balance['dels [m3]']/balance.index.to_series().diff().dt.days

    return balance

pretty_names = {
    'Lam_Pao': 'Lam Pao',
    'Sirindhorn': 'Sirindhorn',
    'Ubol_Ratana': 'Ubol Ratana'
}

def plot_forecast(balance, r, plot_history=30*6):
    latest_nowcast = balance.loc[balance['nowcast/forecast']=='nowcast'].index[-1]
    plot_history_from = latest_nowcast - pd.DateOffset(plot_history)
    forecast_date = balance.index[-1]
    
    f, (ax1, ax2) = plt.subplots(figsize=(10, 7), nrows=2, sharex=True);
    
    ax1.set_title(f"Forecasted ∆S using Sat. derived Rule Curve - {pretty_names[r]}")
    ax1.axvline(latest_nowcast, color='blue', alpha=0.8, linestyle='--', label='Latest Observation')
    (balance.loc[:, 'dels [m3]'] * 1e-6).plot(ax=ax1, color='k', label='∆S [Mil. m3]');
    (balance.loc[:latest_nowcast+pd.DateOffset(1), 'dels [m3]'] * 1e-6).plot(ax=ax1, linewidth=0, label='Nowcast', marker='o', markeredgecolor='blue', markerfacecolor='white', markeredgewidth=2);
    (balance.loc[latest_nowcast+pd.DateOffset(1):, 'dels [m3]'] * 1e-6).plot(ax=ax1, linewidth=0, label='Forecast', marker='o', markeredgecolor='orange', markerfacecolor='white', markeredgewidth=2);
    ax1.set_ylabel('∆S [Mil. m$^3$]')
    ax1.legend(loc='upper left')
    ax1.grid(alpha=0.3)
    ax1.set_xlim(plot_history_from, forecast_date+pd.DateOffset(10))
    ax1.axhline(0, color='k', alpha=0.3, linestyle='--')
    
    
    ax2.set_title(f"Forecasted Outflow - {pretty_names[r]}")
    ax2.axvline(latest_nowcast, color='blue', alpha=0.8, linestyle='--', label='Latest Observation')
    (balance.loc[:, 'O [m3/day]'] * 1e-6).plot(ax=ax2, color='k', label='Outflow [Mil. m3/day]');
    (balance.loc[:latest_nowcast+pd.DateOffset(1), 'O [m3/day]'] * 1e-6).plot(ax=ax2, linewidth=0, label='Nowcast', marker='o', markeredgecolor='blue', markerfacecolor='white', markeredgewidth=2);
    (balance.loc[latest_nowcast+pd.DateOffset(1):, 'O [m3/day]'] * 1e-6).plot(ax=ax2, linewidth=0, label='Forecast', marker='o', markeredgecolor='orange', markerfacecolor='white', markeredgewidth=2);
    ax2.set_ylabel('Outflow [Mil. m$^3$/day]')
    ax2.legend(loc='upper left')
    ax2.grid(alpha=0.3)
    ax2.set_xlim(plot_history_from, forecast_date+pd.DateOffset(10))
    ax2.set_ylim(0, None)
    
    ax2.xaxis.set_major_locator(mdates.MonthLocator(interval=1))
    ax2.xaxis.set_minor_locator(mdates.MonthLocator(interval=1))
    ax2.xaxis.set_major_formatter(mdates.DateFormatter('%b-%Y'))
    for label in ax2.get_xticklabels(which='major'):
        label.set(rotation=0, horizontalalignment='center')
    ax2.set_xlabel('')
    
    f.tight_layout()
    
    return f

In [10]:
@interact(res=[('Lam Pao', 'Lam_Pao'), ('Sirindhorn', 'Sirindhorn'), ('Ubol Ratana', 'Ubol_Ratana')])
def forecast_using_rc(res):
    balance = get_forecast_balance(res, lead_time=15)
    plot_forecast(balance, res)

interactive(children=(Dropdown(description='res', options=(('Lam Pao', 'Lam_Pao'), ('Sirindhorn', 'Sirindhorn'…

# (2) Using manual input of ∆S

In [13]:
@interact(res=[('Lam Pao', 'Lam_Pao'), ('Sirindhorn', 'Sirindhorn'), ('Ubol Ratana', 'Ubol_Ratana')])
def forecast_manual_control(res):
    r = res
    lead_time = 15
    freq = 5

    inflow = get_inflow(r)
    dels_nowcast = get_dels(r)

    dels_nowcast['nowcast/forecast'] = 'nowcast'
    extended_index = pd.date_range(dels_nowcast.index[-1], freq=f"{freq}d", periods=lead_time/freq + 1)[1:]
    
    style = {'description_width': 'initial'}  # https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Styling.html
    a = widgets.FloatText(-9999, description=f'{extended_index[0].strftime("%Y-%m-%d")} (% change w.r.t. SMax ({Smaxes[r]*1e-6} MCM))', style=style, layout=Layout(width='400px'))
    b = widgets.FloatText(-9999, description=f'{extended_index[1].strftime("%Y-%m-%d")} (% change w.r.t. SMax ({Smaxes[r]*1e-6} MCM))', style=style, layout=Layout(width='400px'))
    c = widgets.FloatText(-9999, description=f'{extended_index[2].strftime("%Y-%m-%d")} (% change w.r.t. SMax ({Smaxes[r]*1e-6} MCM))', style=style, layout=Layout(width='400px'))
    ui = widgets.VBox([a, b, c])
    ui.layout.width='500px'
    
    def calculate(t1, t2, t3):
        dels_values = [(s_smax/100)*Smaxes[r] for s_smax in [t1, t2, t3] if s_smax!=-9999]
        dels_forecast = pd.DataFrame({
            'dels [m3]': dels_values,
        }, index=extended_index[:len(dels_values)])
        dels_forecast['nowcast/forecast'] = 'forecast'

        dels = pd.concat([dels_nowcast, dels_forecast])

        balance = dels.copy()

        balance['I [m3]'] = balance['dels [m3]'].rolling(2).apply(lambda x: adder(x, inflow['I [m3]']))

        balance['O [m3]'] = balance['I [m3]'] - balance['dels [m3]']
        balance.loc[balance['O [m3]']<0, 'O [m3]'] = 0
        balance['O [m3/day]'] = balance['O [m3]']/balance.index.to_series().diff().dt.days
        balance['dels [m3/day]'] = balance['dels [m3]']/balance.index.to_series().diff().dt.days
        balance['I [m3/day]'] = balance['I [m3]']/balance.index.to_series().diff().dt.days
        balance['O [Mil. m3/day]'] = balance['O [m3]']*1e-6/balance.index.to_series().diff().dt.days
        balance['dels [Mil. m3/day]'] = balance['dels [m3]']*1e-6/balance.index.to_series().diff().dt.days
        balance['I [Mil. m3/day]'] = balance['I [m3]']*1e-6/balance.index.to_series().diff().dt.days
        print(balance[['I [Mil. m3/day]', 'dels [Mil. m3/day]', 'O [Mil. m3/day]', 'nowcast/forecast']].tail())

        f = plot_forecast(balance, r)
    out = widgets.interactive_output(calculate, {'t1': a, 't2': b, 't3': c})
    
    print("Note: -9999 will be treated as NoData")
    display(ui, out)

interactive(children=(Dropdown(description='res', options=(('Lam Pao', 'Lam_Pao'), ('Sirindhorn', 'Sirindhorn'…

Note: -9999 will be treated as NoData


Output()